In [ ]:
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.datasets import load_digits
import graphviz
from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from time import strptime
import seaborn as sns

In [ ]:
df = pd.read_csv('flag.data')
df

,name,landmass,zone,area,population,language,religion,bars,stripes,colours,...,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright
0,Afghanistan,5,1,648,16,10,2,0,3,5,...,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,...,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,...,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,...,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,...,0,0,0,0,0,0,0,0,blue,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Western-Samoa,6,3,3,0,1,1,0,0,3,...,0,1,5,0,0,0,0,0,blue,red
190,Yugoslavia,3,1,256,22,6,6,0,3,4,...,0,0,1,0,0,0,0,0,blue,red
191,Zaire,4,2,905,28,10,5,0,0,4,...,0,0,0,0,0,1,1,0,green,green
192,Zambia,4,2,753,6,10,5,3,0,4,...,0,0,0,0,0,0,1,0,green,brown


Na podstawie powyższych danych będę próbował przewidzieć religię kraju przy użyciu symboli obecnych na flagach i języka, którym posługuje się ludność danego kraju.

In [ ]:
df.describe()

,landmass,zone,area,population,language,religion,bars,stripes,colours,red,...,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,...,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,3.572165,2.211340,700.046392,23.268041,5.340206,2.190722,0.453608,1.551546,3.463918,0.788660,...,0.170103,0.149485,0.092784,0.149485,1.386598,0.056701,0.139175,0.252577,0.201031,0.082474
std,1.553018,1.308274,2170.927932,91.934085,3.496517,2.061167,1.038339,2.328005,1.300154,0.409315,...,0.463075,0.385387,0.290879,0.435860,4.396186,0.231869,0.347025,0.435615,0.401808,0.275798
min,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,9.000000,0.000000,2.000000,1.000000,0.000000,0.000000,3.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,2.000000,111.000000,4.000000,6.000000,1.000000,0.000000,0.000000,3.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,4.000000,471.250000,14.000000,9.000000,4.000000,0.000000,3.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.750000,0.000000,0.000000
max,6.000000,4.000000,22402.000000,1008.000000,10.000000,7.000000,5.000000,14.000000,8.000000,1.000000,...,4.000000,2.000000,1.000000,4.000000,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.info() #jedynie 4 wartości nie są liczbami, nie ma to większego znaczenia, gdyż będę przewidywał na podstawie symbolii samej obecności kolorów, a nie ich położenia

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        194 non-null    object
 1   landmass    194 non-null    int64 
 2   zone        194 non-null    int64 
 3   area        194 non-null    int64 
 4   population  194 non-null    int64 
 5   language    194 non-null    int64 
 6   religion    194 non-null    int64 
 7   bars        194 non-null    int64 
 8   stripes     194 non-null    int64 
 9   colours     194 non-null    int64 
 10  red         194 non-null    int64 
 11  green       194 non-null    int64 
 12  blue        194 non-null    int64 
 13  gold        194 non-null    int64 
 14  white       194 non-null    int64 
 15  black       194 non-null    int64 
 16  orange      194 non-null    int64 
 17  mainhue     194 non-null    object
 18  circles     194 non-null    int64 
 19  crosses     194 non-null    int64 
 20  saltires  

In [ ]:
df.isna().sum()

name          0
landmass      0
zone          0
area          0
population    0
language      0
religion      0
bars          0
stripes       0
colours       0
red           0
green         0
blue          0
gold          0
white         0
black         0
orange        0
mainhue       0
circles       0
crosses       0
saltires      0
quarters      0
sunstars      0
crescent      0
triangle      0
icon          0
animate       0
text          0
topleft       0
botright      0
dtype: int64

In [ ]:
df.columns

Index(['name', 'landmass', 'zone', 'area', 'population', 'language',
       'religion', 'bars', 'stripes', 'colours', 'red', 'green', 'blue',
       'gold', 'white', 'black', 'orange', 'mainhue', 'circles', 'crosses',
       'saltires', 'quarters', 'sunstars', 'crescent', 'triangle', 'icon',
       'animate', 'text', 'topleft', 'botright'],
      dtype='object')

In [ ]:
X = df[['language','bars','stripes','colours','red', 'green', 'blue',
       'gold', 'white', 'black', 'orange','circles', 'crosses',
       'saltires', 'quarters', 'sunstars', 'crescent', 'triangle', 'icon',
       'animate', 'text']]
X

,language,bars,stripes,colours,red,green,blue,gold,white,black,...,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text
0,10,0,3,5,1,1,0,1,1,1,...,0,0,0,0,1,0,0,1,0,0
1,6,0,0,3,1,0,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0
2,8,2,0,3,1,1,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0
3,1,0,0,5,1,0,1,1,1,0,...,0,0,0,0,0,0,1,1,1,0
4,6,3,0,3,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,1,0,0,3,1,0,1,0,1,0,...,0,0,0,1,5,0,0,0,0,0
190,6,0,3,4,1,0,1,1,1,0,...,0,0,0,0,1,0,0,0,0,0
191,10,0,0,4,1,1,0,1,0,0,...,1,0,0,0,0,0,0,1,1,0
192,10,3,0,4,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
y = df.loc[:,'religion']
y

0      2
1      6
2      2
3      1
4      0
      ..
189    1
190    6
191    5
192    5
193    5
Name: religion, Length: 194, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25) #podział na zbiór testowy i treningowy

In [ ]:
X_train

,language,bars,stripes,colours,red,green,blue,gold,white,black,...,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text
125,3,0,3,3,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
74,6,1,2,4,1,1,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,5,1,0,1,1,1,0,...,0,0,0,0,0,0,1,1,1,0
28,10,0,0,3,1,0,1,0,1,0,...,0,0,0,1,14,0,0,1,1,0
152,10,0,0,2,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,3,0,2,3,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
65,4,0,3,3,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
42,2,0,5,3,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
161,1,5,0,4,0,1,1,1,1,0,...,0,0,0,0,0,0,0,1,1,1


In [ ]:
scaler = MinMaxScaler() #wyskalowanie danych

In [ ]:
scaler.fit(X_train)

MinMaxScaler()

Użyję klasyfikacji z uwagi na to, że przeiwduję religię. W powyższym zbiorze danych jest tylko 8 opcji spośród których można 'wybrać' religię, więc można założyć, że religia podana w zbiorze danych jest dominującą w danym kraju i taką też model ma przewidzieć.

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform (X_test)

In [ ]:
xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [ ]:
XGBClassifier()

XGBClassifier()

In [ ]:
y_pred = xgbc.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 8,  0,  0,  0,  0,  1,  0,  0],
       [ 0, 12,  1,  0,  0,  2,  0,  0],
       [ 0,  1,  8,  0,  0,  3,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  1],
       [ 0,  1,  1,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  0,  0,  2,  0,  0],
       [ 0,  1,  1,  1,  0,  1,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='micro') #f1_score wstępnie wynosi 0,65 co nie jest zadowalającym wynikiem. Nalezy zauważyć, że choć model rzeczywiście wykrywa korelacje między wprowadzonymi danymi to jest daleki od ideału
#parametr average = micro liczy metrykę globalnie poprzez zliczenie true positives, dalse negatives i false positives

0.6326530612244898

In [ ]:
params = dict()
params['n_estimators'] = [300, 700]
params['max_depth'] = [10, 15]
params['max_leaves'] = [0, 10]
params['gamma'] = [1,5]
params['max_delta_step'] = [0,5]

In [ ]:
search = RandomizedSearchCV(xgbc, params)
results = search.fit(X_train, y_train)
best = results.best_estimator_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
best #parametry po tuningu

XGBClassifier(gamma=1, max_depth=10, max_leaves=0, n_estimators=300,
              objective='multi:softprob')

In [ ]:
posttune = best.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, posttune)

array([[ 8,  0,  0,  0,  0,  1,  0],
       [ 1, 12,  1,  0,  0,  1,  0],
       [ 1,  2,  6,  0,  0,  3,  0],
       [ 0,  1,  1,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  0],
       [ 1,  1,  1,  0,  0,  1,  0],
       [ 2,  1,  1,  1,  0,  0,  0]])

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, posttune,average='micro') #f1_score  wynosi 0,69, co jest lepszym wynikiem niż ten uzyskany przed tuningiem, ale wciąż dalkim do ideału

0.5510204081632653

In [ ]:
test = X

In [ ]:
test = scaler.transform(test)

In [ ]:
wynik = best.predict(test)

In [ ]:
X_test_pred = pd.DataFrame(scaler.inverse_transform(X), columns=X.columns, index=y.index)
X_test_pred['religion'] = pd.DataFrame(y)
X_test_pred['religion_pred'] = best.predict(test)
X_test_pred

#porównanie faktycznych religii w poszczególnych krajach z religią przewidzianą

,language,bars,stripes,colours,red,green,blue,gold,white,black,...,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,religion,religion_pred
0,91.0,0.0,39.0,36.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,50.0,0.0,0.0,1.0,0.0,0.0,2,5
1,55.0,0.0,0.0,22.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,50.0,0.0,0.0,0.0,1.0,0.0,6,1
2,73.0,10.0,0.0,22.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,50.0,1.0,0.0,0.0,0.0,0.0,2,2
3,10.0,0.0,0.0,36.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1,1
4,55.0,15.0,0.0,22.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,10.0,0.0,0.0,22.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,4.0,250.0,0.0,0.0,0.0,0.0,0.0,1,1
190,55.0,0.0,39.0,29.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,6,6
191,91.0,0.0,0.0,29.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,5,5
192,91.0,15.0,0.0,29.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5,5


Model w większości przewiduje religię dominującą w danym kraju, choć zdarzają mu się potknięcia. Myślę, że z pomocą tego zbioru danych nie można ulepszyć modelu, gdyż najistotniejsze dane są już wykorzystane przy jego tworzeniu. Co do pominiętych kolumn, uznałem je za zbędne z powodu ich domniemanej niskiej istotności. Potwierdziłem to, podstawiając do modelu jeszcze kolumny 'landmass' i 'zone', które nie tylko nie poprawiły wyników, ale na domiar złego je pogorszyły, zdecydowałem się więc na ich usunięcie.
Po analizie otrzymanych wyników dochodzę do wniosku, że obecność poszczególnych symboli i kolorów na fladze danego państwa może posłużyć do przewidzenia dominującej w nim religii, aczkolwiek nie jest to na tyle dokładny model, żeby dało się ustrzec błędów. Problem pojawia się przy państwach wielokulturowych, tudzież wieloetnicznych. W jednej z iteracji, Islamski Afganistan został przewidziany jako państwo wyznające lokalną religię. Może to być związane z problemem z klasyfikacją poszczególnych odłamów islamu w tym państwie i jemu podobnym np. w Iraku, czy Iranie, gdzie społeczeństwo co prawda wyznaje islam, ale jest mniej więcej równo podzielone pomiędzy sunnitów i szyitów.